because are dataset is already denoised, they are already truncated & we don't need to know wheter they are paired or single end reads because they are already assembled

### Classifiying

In [1]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

import qiime2 as q2

%matplotlib inline


Data = 'Data'

In [8]:
# 1. Import reference sequences
!qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path Data/UNITE_dynamic/sh_refs_qiime_ver10_dynamic_19.02.2025.fasta \
  --output-path Data/processed/unite-ref-seqs.qza

# 2. Import taxonomy
!qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path Data/UNITE_dynamic/sh_taxonomy_qiime_ver10_dynamic_19.02.2025.txt \
  --output-path Data/processed/unite-ref-taxonomy.qza

# 3. Train the classifier
#!qiime feature-classifier fit-classifier-naive-bayes \
#  --i-reference-reads Data/processed/unite-ref-seqs.qza \
#  --i-reference-taxonomy Data/processed/unite-ref-taxonomy.qza \
#  --o-classifier Data/processed/unite-dynamic-classifier.qza

! qiime feature-classifier fit-classifier-naive-bayes \
    --i-reference-reads $Data/processed/unite-ref-seqs.qza \
    --i-reference-taxonomy $Data/processed/unite-ref-taxonomy.qza \
    --p-classify--chunk-size 2000 \
    --o-classifier $Data/processed/unite-dynamic-classifier.qza
    

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Imported Data/UNITE_dynamic/sh_refs_qiime_ver10_dynamic_19.02.2025.fasta as DNASequencesDirectoryFormat to Data/processed/unite-ref-seqs.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Imported Data/UNITE_dynamic/sh_taxonomy_qiime_ver10_dynamic_19.02.2025.txt as HeaderlessTSVTaxonomyFormat to Data/processed/unite-ref-taxonomy.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__

dereplication for lower Mem use

In [18]:
! qiime rescript dereplicate \
    --i-sequences Data/processed/unite-ref-seqs.qza \
    --i-taxa Data/processed/unite-ref-taxonomy.qza \
    --p-threads 3 \
    --p-mode 'lca' \
    --o-dereplicated-sequences Data/processed/unite-ref-seqs-lca.qza \
    --o-dereplicated-taxa  Data/processed/unite-ref-taxonomy-lca.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Sequence] to: Data/processed/unite-ref-seqs-lca.qza
Saved FeatureData[Taxonomy] to: Data/processed/unite-ref-taxonomy-lca.qza


remove sequences with unhelpful taxonomy to reduce size

In [30]:
! qiime taxa filter-seqs \
    --p-include Fungi_sp \
    --p-exclude "Unassigned,Ambiguous_taxa,Incertae_sedis,Chloroplast,Mitochondria,Viridiplantae,Metazoa,Alveolata" \
    --i-taxonomy Data/processed/unite-ref-taxonomy-lca.qza\
    --i-sequences Data/processed/unite-ref-seqs-lca.qza \
    --o-filtered-sequences Data/processed/unite-ref-seqs-lca-filtered.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Plugin error from taxa:

  All features were filtered, resulting in an empty collection of feature sequences.

Debug info has been saved to /tmp/qiime2-q2cli-err-9qygxcl0.log


In [34]:
!qiime metadata tabulate \
  --m-input-file Data/processed/unite-ref-taxonomy-lca.qza \
  --o-visualization Data/processed/taxonomy-summary.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Data/processed/taxonomy-summary.qzv


this shows that these are all already Fungis on species level and I don't know which we can kick out on lower level; we should not just use the Fungal DB because then misclassification of host DB and others is quite high (if you just train on fungal ITS, everything starts to look like fungal
no singleton

In [27]:
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads Data/processed/unite-ref-seqs-lca.qza \
  --i-reference-taxonomy Data/processed/unite-ref-taxonomy-lca.qza\
  --p-classify--chunk-size 10 \
  --o-classifier Data/processed/unite-dynamic-classifier.qza 

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


I think above method uses too much memory, because always aborted when Mem red

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier 'Data/processed/unite-dynamic-classifier.qza' \
  --i-reads 'Data/processed/rep-seqs.qza' \
  --o-classification taxonomy.qza

the last file (classifier) is not created; maybe to small of RAM, check with week 5 where we did it with several parallel

Citation of one UNITE classifier
Abarenkov, Kessy; Zirk, Allan; Piirmann, Timo; Pöhönen, Raivo; Ivanov, Filipp; Nilsson, R. Henrik; Kõljalg, Urmas (2025): UNITE QIIME release for Fungi. UNITE Community. 10.15156/BIO/3301241